In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
train_loc='hackerearth Tis STILL the season to be jolly/train'
test_loc='hackerearth Tis STILL the season to be jolly/test'
df='hackerearth Tis STILL the season to be jolly/train.csv'

In [7]:
data=pd.read_csv('hackerearth Tis STILL the season to be jolly/train.csv')

In [8]:
data.head()

,Image,Class
0,image3476.jpg,Miscellaneous
1,image5198.jpg,Candle
2,image4183.jpg,Snowman
3,image1806.jpg,Miscellaneous
4,image7831.jpg,Miscellaneous


In [5]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3

In [26]:
train_datagen=ImageDataGenerator(rescale=1/255,shear_range=.1,horizontal_flip=True)


In [27]:
train_generator=train_datagen.flow_from_dataframe(data,train_loc,'Image','Class',target_size=(224,224),class_mode='categorical',shuffle=True,batch_size=64)


Found 6469 validated image filenames belonging to 6 classes.
Found 0 validated image filenames belonging to 0 classes.


C:\Users\ajay\Anaconda3\lib\site-packages\keras_preprocessing\image\dataframe_iterator.py:273: UserWarning: Found 6469 invalid image filename(s) in x_col="Image". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [34]:
train_generator.class_indices

{'Airplane': 0,
 'Candle': 1,
 'Christmas_Tree': 2,
 'Jacket': 3,
 'Miscellaneous': 4,
 'Snowman': 5}

In [35]:
label={0:'Airplane',1:'Candle',2:'Christmas_Tree',3:'Jacket',4:'Miscellaneous',5:'Snowman'}

In [28]:
from keras.models import Model
from keras.layers import Dense,Flatten

In [15]:
inception=InceptionV3(weights='imagenet',input_shape=(224,224,3),include_top=False)

87916544/87910968 [==============================] - 127s 1us/step


In [17]:
for layer in inception.layers:
    layer.trainable=False

In [18]:
x=Flatten()(inception.output)
out=Dense(6,activation='softmax')(x)

In [19]:
model=Model(inputs=inception.input,outputs=out)

In [20]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 111, 111, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 111, 111, 32) 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

conv2d_38 (Conv2D)              (None, 12, 12, 128)  114688      activation_37[0][0]              
__________________________________________________________________________________________________
batch_normalization_33 (BatchNo (None, 12, 12, 128)  384         conv2d_33[0][0]                  
__________________________________________________________________________________________________
batch_normalization_38 (BatchNo (None, 12, 12, 128)  384         conv2d_38[0][0]                  
__________________________________________________________________________________________________
activation_33 (Activation)      (None, 12, 12, 128)  0           batch_normalization_33[0][0]     
__________________________________________________________________________________________________
activation_38 (Activation)      (None, 12, 12, 128)  0           batch_normalization_38[0][0]     
__________________________________________________________________________________________________
average_po

In [23]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [29]:
model.fit_generator(train_generator,epochs=8,steps_per_epoch=len(train_generator))

Epoch 1/8
102/102 [==============================] - 1227s 12s/step - loss: 0.7435 - accuracy: 0.8366
Epoch 2/8
102/102 [==============================] - 1171s 11s/step - loss: 0.6073 - accuracy: 0.8664
Epoch 3/8
102/102 [==============================] - 1169s 11s/step - loss: 0.6452 - accuracy: 0.8728
Epoch 4/8
102/102 [==============================] - 1184s 12s/step - loss: 0.9281 - accuracy: 0.8658
Epoch 5/8
102/102 [==============================] - 1330s 13s/step - loss: 0.5801 - accuracy: 0.8957
Epoch 6/8
102/102 [==============================] - 1251s 12s/step - loss: 0.4869 - accuracy: 0.9077
Epoch 7/8
102/102 [==============================] - 1208s 12s/step - loss: 0.3532 - accuracy: 0.9317
Epoch 8/8
102/102 [==============================] - 1169s 11s/step - loss: 0.3799 - accuracy: 0.9287


In [30]:
model.save('christmas_holiday.h5')

In [36]:
preds=model.predict_generator(train_generator)

In [38]:
k=np.argmax(preds,axis=1)

In [57]:
prediction=[]
for i in k:
    prediction.append(label[i])
    

In [58]:
from sklearn.metrics import f1_score,accuracy_score

In [59]:
f_score=f1_score(data['Class'],prediction,average='weighted')
acc_score=accuracy_score(data['Class'],prediction)


In [61]:
acc_score

0.25490802287834285

In [62]:
submission=pd.DataFrame([])

In [68]:
j=0
for i in os.listdir(test_loc):
    print(i)
    j+=1
    if j>10:
        break

image10.jpg
image100.jpg
image1013.jpg
image1014.jpg
image1018.jpg
image1019.jpg
image1021.jpg
image1023.jpg
image1024.jpg
image1029.jpg
image1030.jpg


In [69]:
li=[]
for i in os.listdir(test_loc):
    li.append(i)
submission['Image']=li

In [73]:
submission.head()

,Image
0,image10.jpg
1,image100.jpg
2,image1013.jpg
3,image1014.jpg
4,image1018.jpg


In [76]:
test_datagen=ImageDataGenerator(rescale=1/255)
test_generator=test_datagen.flow_from_dataframe(
dataframe=submission,
directory=test_loc,
x_col="Image",
y_col=None,
batch_size=64,
seed=42,
shuffle=False,
class_mode=None,
target_size=(224,224))

Found 3489 validated image filenames.


In [77]:
test_pred=np.argmax(model.predict_generator(test_generator),axis=1)
test_prediction=[]
for i in test_pred:
    test_prediction.append(label[i])
    

In [79]:
submission['Class']=test_prediction

In [81]:
submission.to_csv('christmas.csv',index=None)